<h1><center> Network/Graph Analysis in Python </center></h1>

**NetworkX**: Python package for the creation, manipulation, and study of the structure, dynamics, and functions of complex networks.

Installation: 
`$ pip install networkx`

Version check in Python:
`networkx.__version__`

Upgrade:
`$ pip install networkx --upgrade`

In [ ]:
import numpy as np
import pandas as pd
from collections import Counter
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# check version
nx.__version__

**Some open source network data locations**:
* **The Koblenz Network Collection**: http://konect.uni-koblenz.de/
* **Stanford Large Network Dataset Collection**: https://snap.stanford.edu/data/

## I. Networks Basics

### 1. Create & Manipulate Networks 

Let's start with simple undirected and unweighted networks. An example of such networks, where we don't have edge direction (in/out) or edge weight (indicating how strong a connection is), would be the Facebook network. If you are a friend of mine, I am a friend of yours, and the edge connecting us is either 1 (connected) or non-existent (not connected). We will generate more complex networks later on.

**Nodes**: can represent anything (images, webpage URL links, people, power stations, numbers, etc.)

**Edges**: represent relationships between nodes

In [ ]:
# create empty network
G = nx.Graph()

In [ ]:
# add one node with label '1'
G.add_node('Mary')

In [ ]:
# add nodes from a list of elements
G.add_nodes_from(['Mary', 'Steven', 'Alice'])

In [ ]:
# remove node
G.remove_node('Mary')

In [ ]:
# remove multiple nodes
G.remove_nodes_from(['Mary', 'Steven'])

In [ ]:
# view nodes in network G
G.nodes

In [ ]:
# add single edge - tuple of nodes (source, target)
# this also adds nodes if they don't already exist
G.add_edge('Mary','Steven')

In [ ]:
# add multiple of edges (list of tuples)
G.add_edges_from([('Mary', 'Steven') , ('Mary', 'Alice')])

In [ ]:
# view edges in network G
G.edges

In [ ]:
# remove edge
G.remove_edge('Mary','Alice')

In [ ]:
# remove multiple edges (list of tuples)
G.remove_edges_from([('Mary', 'Steven') , ('Mary', 'Alice')])

In [ ]:
# get number of nodes in network G
G.number_of_nodes()

In [ ]:
# get number of edges in network G
G.number_of_edges()

In [ ]:
# get Alice's neighbors (retrieves a dictionary)
dict_neighbors = G.neighbors('Alice')

In [ ]:
# get Alice's number of neighbors (connections)
G.degree('Alice')

In [ ]:
# empty the network
G.clear()

**Load network from file.** You can read/write a graph in a file using common graph formats (edge lists, adjacency lists, GML, GraphML, pickle, LEDA, etc.).

In [ ]:
G = nx.read_edgelist("../datafiles/social/facebook/facebook_combined.txt")

In [ ]:
# output edgelist to file
nx.write_edgelist(fb,'fb.edgelist',data=False)

In [ ]:
df1.to_csv("fb_edges_only.txt", sep=' ', header=False, index=False)

In [ ]:
fb = nx.read_edgelist("../datafiles/social/facebook/fb_edges_only.txt")

In [ ]:
len(fb.nodes)

In [ ]:
nx.degree_assortativity_coefficient(fb)

In [ ]:
nx.degree_pearson_correlation_coefficient(fb)

In [ ]:
gnutella = nx.read_edgelist("p2p-Gnutella08.txt")

In [ ]:
nx.degree_pearson_correlation_coefficient(gnutella)

In [ ]:
# read edgelist
G = nx.read_edgelist("test.edgelist")

# write edgelist
nx.write_edgelist(G, "test.edgelist")

### 2. Network Types 

#### a. Weighted Graphs

**Edge weight.** Consider that the edge that you are adding should contain additional information, such as the strength of the connection. This would be important, for example, when analyzing communication networks to check friendship/connectivity strength. You want to capture how many times they exchanged e-mails, calls, text messages, to indicate the strength of the connection. For this you will assign weights to the edge, values that can be the number of communications, or the fraction of communications, normalized.

In [ ]:
# assign weight to edge
G.add_edge('Mary','Steven', weight=5)

In [ ]:
# access edge weight
G['Mary']['Steven']

In [ ]:
# change edge weight
G['Mary']['Steven']['weight'] = 6

#### b. Directed Graphs

**Edge direction.** Edges have direction describing source -> target node relationship.

In [ ]:
dg = nx.DiGraph()

In [ ]:
# you can create an undirected representation of network G
nx.to_undirected(G)

In [ ]:
# you can create a directed representation of network G
nx.to_directed(G)

#### c. Multigraphs

Many algorithms are not well defined on such graphs. Therefore, you should convert such graphs rather to a standard graph in a way that makes the measurement well defined.

In [ ]:
# multigraphs can store multiple edges information between same two nodes that can have different properties
MG = nx.MultiGraph()
MG.add_weighted_edges_from([(1, 2, 3.0), (1, 2, 75), (2, 3, 5)])

In [ ]:
# shows the edges with no weights
MG.edges

In [ ]:
# shows the weights of the edges as well
MG.edges.data('weight', default=1)

In [ ]:
# check the weight of an edge
MG[1][2]

### 3. Network Models

There are a miriad of network models with different topological properties. Here we will try out some of the most useful ones (that frequently occur in real complex systems). 

For more network generation classes: https://networkx.github.io/documentation/networkx-1.10/reference/generators.html

In [ ]:
# Barabasi-Albert (scale-free) network 
ba = nx.barabasi_albert_graph(50, 3)

In [ ]:
nx.draw_spectral(ba)

In [ ]:
# Erdos-Renyi (random) network 
er = nx.erdos_renyi_graph(50, 0.1)

In [ ]:
nx.draw_circular(er)

In [ ]:
# Watts-Strogatz (small-world) network 
ws = nx.watts_strogatz_graph(50, 6, 0.2)

In [ ]:
nx.draw_circular(ws)

In [ ]:
# random geometric graph (RGG)
rgg = nx.random_geometric_graph(200,0.125)

In [ ]:
nx.draw(rgg)

In [ ]:
# complete graph (every pair of nodes is connected by a unique edge)
complete = nx.complete_graph(6)

In [ ]:
nx.draw(complete)

## II. Analysis of Structural Properties

#### 1. Node degree, network average degree, degree distribution

In [ ]:
degrees = [deg for node, deg in nx.degree(G)]

In [ ]:
# kmin - minimum degree
kmin = np.min(degrees)

# kmax - maximum degree
kmax = np.max(degrees)

# kavg - average degree of the network
kavg = np.mean(degrees)

**Degree distribution**: helps us understand connectivity trends in networks and how edges are distributed among nodes (does everyone have similar number of connections, or do we have hubs, nodes with significantly higher number of connections?)

In [ ]:
def degree_distr(net):
    degrees = dict(net.degree()) 
    hist = list(Counter(degrees.values()).items()) 
    hist.sort(key=lambda x:x[0])
    hist = np.array(hist)
    return hist

#### 2. Paths on Networks: average path length, shortest path, longest path

In [ ]:
# find shortest path in directed & undirected network
nx.shortest_path(G,'b','d')
nx.shortest_path(g,'b','d', weighted=True)

#### 3. Clustering coefficient, triangles

In [ ]:
# triangles
nx.triangles(G)

In [ ]:
# clustering coefficient of a node
nx.clustering(G, 'Mary')

In [ ]:
# clustering coefficient of all nodes (returns a dictionary)
nx.clustering(G)

In [ ]:
# clustering coefficient of the network
cc = nx.clustering(G)
avg_clust = sum(cc.values()) / len(cc)
print(avg_clust)

#### 4. Centrality measures

In [ ]:
# degree centrality
nx.degree_centrality(G)

In [ ]:
# betweenness centrality of network
nx.betweenness_centrality(G)

In [ ]:
# closeness centrality of network
nx.closeness_centrality(G)

In [ ]:
# eigenvector centrality of network
nx.eigenvector_centrality(G)

#### 5. Components

In [ ]:
# checks whether the network is connected
nx.is_connected(G)

In [ ]:
# find number of connected components
nx.number_connected_components(G)

In [ ]:
# get the nodes in the same component as *n*
nx.node_connected_component(G, 'Mary')

#### 6. Assortativity

* Pearson correlation coefficient [-1; 1]
* social networks are highly assortative (homophily): high degree nodes connect to other high degree nodes
* technological are disassortative: high degree nodes connect to low degree nodes

Assortativity computed based on:
* degree
* attribute

In [ ]:
nx.degree_assortativity_coefficient(ba)

In [ ]:
nx.degree_pearson_correlation_coefficient(ba)

In [ ]:
# check assortativity (mixing) by a particular attribute
nx.attribute_assortativity_coefficient(ba, attribute)

## III. Network Visualization

What you've all been waiting for! :)

Disclaimer: visualizations can be pretty and insightful, however for large networks they tend to be just pretty! Knowing how to compute network measures is the key!

* visualization with NetworkX: *"NetworkX provides basic functionality for visualizing graphs, but its main goal is to enable graph analysis rather than perform graph visualization. In the future, graph visualization functionality may be removed from NetworkX or only available as an add-on package."*
* Matplotlib or Graphviz with pydot (import and export NetworkX graphs in Graphviz dot format using pydot)
* Gephi
* Graphviz
* Neo4j
* D3
* etc.

Basic drawing methods in NetworkX module: 
* `nx.draw()`
* `nx.random()`
* `nx.draw_spectral()`
* `nx.draw_circular()`

In [ ]:
# simplest way to draw a graph
nx.draw(G)

In [ ]:
# nxviz package provides some nice visualization options
import nxviz as nv

In [ ]:
c = nv.CircosPlot(ba) 
c.draw()

In [ ]:
c = nv.CircosPlot(rgg) 
c.draw()

In [ ]:
c = nv.CircosPlot(er) 
c.draw()

#### 2. Visualize subgraphs

In [ ]:
# assign selected subgraph to a new graph
nodes = list(ba.neighbors(10))
nodes.append(10)
G_sub = ba.subgraph(nodes)

In [ ]:
nx.draw(G_sub, with_labels=True)

## IV. Queries On Networks

* find specific nodes
* find specific edges

In [ ]:
# obtain a list of nodes with a certain property
sub_nodes = [n[0] for n in G.nodes(data=True) if d['attribute'] == 'what we are interested in']

In [ ]:
# obtain a list of edges with a certain property
sub_edges = [edge for edge in G.edges(data=True) if d['attribute'] == 'what we are interested in']

In [ ]:
# find cliques
nx.find_cliques(ba)

In [ ]:
list(nx.find_cliques(ba))

In [ ]:
# assign selected subgraph to a new graph
nodes = list(ba.neighbors(10))
nodes.append(10)
G_sub = ba.subgraph(nodes)

In [ ]:
nx.draw(G_sub, with_labels=True)

## V. Case Studies - Real Network Analysis from Data 

### V. 1. Social Networks
using ***Graph Embedding with Self Clustering: Facebook data*** 

- data location: SNAP (Stanford Large Network Dataset Collection)
- source: B. Rozemberczki, R. Davies, R. Sarkar and C. Sutton. GEMSEC: Graph Embedding with Self Clustering. 2018.

<img src="../images/data.png" alt="Data" style="width: 300px;"/>

* nodes: pages
* edges: mutual likes among them -> this means undirected & networks


There are 8 different networks representing different categories. For our current analysis, we will analyze and compare 4 networks `Company`, `Artist`, `Politician` and `Public Figure`, however, you can play with all of them to get used to manipulating networks and computing structural property measures, and to gain insights about the data.

### a. Load network from file

In [ ]:
# load edgelists from data file
co_net = nx.read_edgelist("facebook_clean_data/company_edges.csv", delimiter=',')
art_net = nx.read_edgelist("facebook_clean_data/artist_edges.csv", delimiter=',')
poli_net = nx.read_edgelist("facebook_clean_data/politician_edges.csv", delimiter=',')
pub_net = nx.read_edgelist("facebook_clean_data/public_figure_edges.csv", delimiter=',')

### b. Analyze network

In [ ]:
# check number of nodes and edges 
N = len(poli_net.nodes)
print(N)

E = len(poli_net.edges)
print(E)

Let's see what is on average the number of mutual 'Likes' (connections) each page (node) has. 

In [ ]:
# calculate average degree of politician pages
degrees = [deg for node, deg in nx.degree(poli_net)]
kavg = np.mean(degrees)
print("The # of mutual likes politician pages have on average:", kavg)

# calculate average degree of artist pages
degrees = [deg for node, deg in nx.degree(art_net)]
kavg = np.mean(degrees)
print("The # of mutual likes artist pages have on average:", kavg)

# calculate average degree of company pages
degrees = [deg for node, deg in nx.degree(co_net)]
kavg = np.mean(degrees)
print("The # of mutual likes company pages have on average:", kavg)

# calculate average degree of public figure pages
degrees = [deg for node, deg in nx.degree(pub_net)]
kavg = np.mean(degrees)
print("The # of mutual likes public figure pages have on average:", kavg)

What about the degree distribution in the networks?

In [ ]:
hist_poli = degree_distr(poli_net)
hist_art = degree_distr(art_net)
hist_co = degree_distr(co_net)
hist_pub = degree_distr(pub_net)

plt.figure()
plt.loglog(hist_poli.T[0],hist_poli.T[1],'ro-')
plt.loglog(hist_art.T[0],hist_art.T[1],'ko-')
plt.loglog(hist_co.T[0],hist_co.T[1],'go-')
plt.loglog(hist_pub.T[0],hist_pub.T[1],'bo-')
plt.legend(['Politician', 'Artist', 'Company', 'Public Figure'])
plt.xlabel('Degree')
plt.ylabel('Number of nodes')
plt.title('Facebook Networks by Category')
plt.savefig('fb_network_degree_distr.png')

In [ ]:
# we can check assortativity
nx.degree_assortativity_coefficient(pub_net)

In [ ]:
# check clustering coefficient
cc = nx.clustering(poli_net)
avg_cc_poli = sum(cc.values()) / len(cc)
print("Politician network clustering coefficient:", avg_cc_poli)

cc = nx.clustering(art_net)
avg_cc_art = sum(cc.values()) / len(cc)
print("Artist network clustering coefficient:", avg_cc_art)

cc = nx.clustering(co_net)
avg_cc_co = sum(cc.values()) / len(cc)
print("Company network clustering coefficient:", avg_cc_co)

cc = nx.clustering(pub_net)
avg_cc_pub = sum(cc.values()) / len(cc)
print("Public figure network clustering coefficient:", avg_cc_pub)

### c. Visualize network


### d. Takeaways

* politician network has highest clustering coefficient (CC = 0.39) -> political views tend to cluster people more densely
* public figure network is has the highest assortativity -> the only network that reflects human relationship type of connectivity
* disassortativity seen in company, politician, artist networks may be indicative of competitiveness, desire not to like/promote other popular peers
* lowest average degree found in company network
* highest average degree seen in artist network
* each network has similar degree distribution with heavy tail -> most nodes have small number of connections, while a few nodes (hubs) have significantly higher number of links
* and many more insights with more detailed analyses -> it is your homework to explore more! :)

In [ ]:
G_phys = nx.read_edgelist("../datafiles/social/physicians/out.moreno_innovation_innovation", comments='%')

In [ ]:
nx.draw(G_phys, node_size=5)

We practiced and tried out different analysis techniques using social networksas case-study. 

**Homework:** load below the infrastructure, technological, biological, lexical networks and analyze their properties! 

### V. 2. Infrastructure Networks

In [ ]:
G_infra = nx.read_edgelist("../datafiles/infrastructure/powergrid/out.opsahl-powergrid", comments='%')

In [ ]:
nx.draw(G_infra, node_size=2, node_color='g')

In [ ]:
len(G_infra.nodes)

In [ ]:
pos = nx.spectral_layout(G_infra)

In [ ]:
nx.draw(G_infra, pos, node_size=2)

### V. 3. Technological Networks

In [ ]:
G_tech = nx.read_edgelist("../datafiles/technological/p2p/p2p-Gnutella08.txt")

### V. 4. Biological Networks

In [ ]:
G_bio = nx.read_edgelist("../datafiles/biological/protein/facebook_combined.txt")

### V. 5. Lexical Networks

In [ ]:
G_lex = nx.read_edgelist("../datafiles/lexical/david_copperfield/out.adjnoun_adjacency_adjacency", comments='%')

In [ ]:
nx.draw(G_lex, with_labels=True, node_size=30, node_color='g', edge_color='grey')

## VI. Bonus: Network Features for Machine Learning Models